<a href="https://colab.research.google.com/github/sgparrot/sgparrot2021Lv1-1/blob/main/MNIST_model(%EA%B9%80%EB%AF%BC%EA%B0%95).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
plt.imshow(X_train[0])


In [ ]:
plt.figure(figsize=(20, 10))
for i in range(9):
  plt.subplot(330+1+i)
  plt.imshow(X_train[i])
plt.show()

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
X_train = X_train.reshape((X_train.shape[0],28,28,1))
X_test = X_test.reshape((X_test.shape[0], 28,28,1))

In [ ]:
#0과 1사이 값으로 Normalization

X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255

In [ ]:
#one-hot 인코딩

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
#여기까지 데이터 전처리

In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3), activation = 'relu', kernel_initializer ='he_uniform', input_shape = (28,28,1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3,3), padding = 'same', activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten()) 
model.add(layers.Dropout(0.5)) #overfitting 방지
model.add(Dense(32, activation = 'relu'))
model.add(Dense(10, activation="softmax"))
  
model.summary()

In [ ]:
#모델 컴파일

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [ ]:
hist = model.fit(X_train, y_train, batch_size=100, epochs = 15, validation_split = 0.2, verbose = 1)

In [ ]:
#시각화


def acc_loss_plot(hist):
  fig, loss_ax=plt.subplots()
  acc_ax=loss_ax.twinx()

  loss_ax.plot(hist.history['loss'], 'y', label='train loss')
  loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
  loss_ax.set_xlabel('epoch')
  loss_ax.set_ylabel('loss')
  loss_ax.legend(loc='upper left')

  acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
  acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')
  acc_ax.set_ylabel('accuracy')
  acc_ax.legend(loc='upper right')

  plt.show()

In [ ]:
print("baseline accuracy")
print(acc_loss_plot(hist))

In [ ]:
score = model.evaluate(X_test, y_test, verbose = 0)
print(score[0]) #test loss
print(score[1]) #test accuracy

In [ ]:
#예측 결과와 실제 값 비교
predicted_result = model.predict(X_test)
predicted_labels = np.argmax(predicted_result, axis = 1)
test_labels = np.argmax(y_test, axis=1)
print(predicted_labels[:10])
print(test_labels[:10])


In [ ]:
#틀린 데이터 개수

wrong_result = []

for n in range(0, len(y_test)):
    if predicted_labels[n] != test_labels[n]:
        wrong_result.append(n)
len(wrong_result)


In [ ]:
#틀린 것 중 랜덤하게 9개 확인

import random

wrong = random.choices(population= wrong_result, k=9)


wrong

In [ ]:
#틀린 데이터 시각화 

plt.figure(figsize=(14, 12))

for idx, n in enumerate(wrong):
    plt.subplot(3, 3, idx + 1)
    plt.imshow(X_test[n].reshape(28, 28), cmap = 'Greys', interpolation='nearest')
    plt.title('True : ' + str(test_labels[n]) + ' Predict : ' + str(predicted_labels[n]))
    plt.axis('off')
    
plt.show()